In [26]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [84]:
def load_data_for_split(file_path):
    texts, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            text = line.replace(' ', '')
            label = []
            i = 0
            while (i < len(text)):
                if text[i] in ['，', '。', '？', '！']:
                    label.append('1')
                else:
                    label.append('2')
                i += 1
            text_list = list(text)
            # label[-1] = 2
            # print(text, label)
            # break
            texts.append(text_list)
            labels.append(''.join(label))

            if (len(text_list) != len(label)):
                print('Error:', text, label)
    return texts, labels

In [85]:
texts, labels = load_data_for_split('data/train_large_2.txt')

In [86]:
print(texts[:5])
print(labels[:5])

[['忽', '忽', '幾', '晨', '昏', '，', '離', '別', '間', '之', '，', '疾', '病', '間', '之', '，', '不', '及', '終', '年', '同', '靜', '好', '。'], ['煢', '煢', '小', '兒', '女', '，', '孱', '羸', '若', '此', '，', '嬌', '憨', '若', '此', '，', '更', '煩', '二', '老', '費', '精', '神', '。'], ['毋', '人', '負', '我', '，', '毋', '我', '負', '人', '，', '柳', '下', '雖', '和', '有', '介', '稱', '，', '先', '生', '字', '此', '，', '可', '以', '諡', '此', '。'], ['愛', '老', '臣', '少', '，', '愛', '少', '臣', '老', '，', '馮', '唐', '爲', '郎', '無', '倦', '意', '，', '吾', '輩', '慕', '之', '，', '不', '能', '效', '之', '。'], ['深', '院', '落', '滕', '花', '，', '石', '不', '點', '頭', '龍', '不', '語', '。']]
['222221222212222122222221', '222221222212222122222221', '2222122221222222212222122221', '2222122221222222212222122221', '22222122222221']


In [106]:
# Calculate the ratio of punctuation marks
bins = [int(label) for label in labels] # Normal
# bins = [int(label, 3) for label in labels] # Ternary
print(bins[:5])

[222221222212222122222221, 222221222212222122222221, 2222122221222222212222122221, 2222122221222222212222122221, 22222122222221]


In [102]:
# Count unique values in bins
unique, counts = np.unique(bins, return_counts=True)
print(len(unique), unique, counts)

2816 [             700              727             2095 ... 1853020188851839
 5535897843252868 5559057079593244] [24  2  4 ...  2  2  6]


In [107]:
# Create a DataFrame for easier manipulation
df = pd.DataFrame({'labels': labels, 'bins': bins, 'texts': texts})

# Step 3: Stratified K-Fold Cross Validation using the binned categories
skf = StratifiedKFold(n_splits=5)


In [72]:
# output df to csv
df.to_csv('data/train_large_bin_2.csv', index=False)

In [114]:
cnt_train = 0
cnt_valid = 0

for i, (train_index, val_index) in enumerate(skf.split(df['texts'], df['bins'])):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]
    
    X_train, y_train = train_df['texts'].tolist(), train_df['bins']
    X_val, y_val = val_df['texts'].tolist(), val_df['bins']

    
    with open(f'data/split3/train_{i}.txt', 'w', encoding='utf-8') as file:
        for text, label in zip(X_train, y_train):
            file.write(''.join(text) + '\n')
            cnt_train += 1

    with open(f'data/split3/valid_{i}.txt', 'w', encoding='utf-8') as file:
        for text, label in zip(X_val, y_val):
            file.write(''.join(text) + '\n')
            cnt_valid += 1

print(cnt_train, cnt_valid)


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'unknown' instead.